In [1]:
import sys
import os

# Add the path to the parent module
sys.path.append(os.path.abspath('../..'))

import warnings
warnings.filterwarnings('ignore')


import deeppy as dp

import torch
import torch.optim as optim
import torch.nn as nn
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


import numpy as np
import matplotlib.pyplot as plt

import tiktoken
from datasets import load_dataset


from deeppy import LearnFrame,LayerGenerator,FromLoader
from deeppy.models.nlp.gpt import GPT
from deeppy import GPTText


In [2]:
encoding = tiktoken.encoding_for_model("gpt-2")


vocab_size = encoding.n_vocab
context_size = 1024

In [3]:
dataset = load_dataset("wikitext", "wikitext-103-v1")


In [6]:
print("".join(dataset["train"]["text"][:100]))

 = Valkyria Chronicles III = 
 Senjō no Valkyria 3 : <unk> Chronicles ( Japanese : 戦場のヴァルキュリア3 , lit . Valkyria of the Battlefield 3 ) , commonly referred to as Valkyria Chronicles III outside Japan , is a tactical role @-@ playing video game developed by Sega and Media.Vision for the PlayStation Portable . Released in January 2011 in Japan , it is the third game in the Valkyria series . Employing the same fusion of tactical and real @-@ time gameplay as its predecessors , the story runs parallel to the first game and follows the " Nameless " , a penal military unit serving the nation of Gallia during the Second Europan War who perform secret black operations and are pitted against the Imperial unit " <unk> Raven " . 
 The game began development in 2010 , carrying over a large portion of the work done on Valkyria Chronicles II . While it retained the standard features of the series , it also underwent multiple adjustments , such as making the game more forgiving for series newcomers . 

In [7]:
encoding.encode("".join(dataset["train"]["text"][:100]))

TypeError: expected string or buffer

In [ ]:

data = GPTText(train=dataset["train"]["text"], tokenizer=encoding, context_size = context_size, test=dataset["test"]["text"], valid = dataset["validation"]["text"])

In [4]:
Optimizer_params = {
    "optimizer":optim.AdamW,
    "optimizer_args":{"lr":3e-4, "amsgrad" : True, "weight_decay" : 0, "betas" : (0.9, 0.95)},
    "clipper":nn.utils.clip_grad_norm_,
    "clipper_params":{"max_norm" : 1.0},
    "scheduler_params":None,
}

GPT_params = {
    "optimizer_params":Optimizer_params,
    "vocab_size":vocab_size,
    "embed_dim":768,
    "num_heads":12,
    "num_layers":12,
    "context_size":context_size,
    "device":device,
    "criterion":nn.CrossEntropyLoss(ignore_index = -1),
}

model = GPT(**GPT_params)

print(f"Total parameters : {sum(p.numel() for p in model.net.parameters()) / 1e6} Million")

Total parameters : 3.267584 Million


In [5]:
lf = LearnFrame(model,data)

In [108]:
epoch = 5000

for i in range(epoch):
    lf.optimize()
    lf.test()

    if i % 100 == 0:
        lf.plot(show_result=False, log=True)
lf.plot(show_result=True, log=True, save = "GPT.png")

KeyboardInterrupt: 

<Figure size 640x480 with 0 Axes>

In [106]:
txt = "On this very beautiful day, let us"
for i in range(10):
    inp = torch.tensor(encoding.encode(txt)).unsqueeze(0)
    out = model.generate(inp, 9).cpu()
    inp = out
    txt += encoding.decode(out.squeeze(0).tolist())
print(txt)

On this very beautiful day, let us.


ABKEES:
And,,


L:

I:
I
I

, the,
I
I,
I
I,
And,
I,

IUS:
IUS,
I, the the,, the,,
And the,

And:
I,:
I,,,,,,,
I the,
